In [166]:
import numpy as np

In [167]:
w1 = 2#@param
w2 = 8#@param
w3 = 4#@param
w4 = 5#@param

c1 = 10#@param
c2 = 4#@param
c3 = 5#@param
c4 = 6#@param

constraint = 15#@param

weights = np.array([w1, w2, w3, w4])
costs = np.array([c1, c2, c3, c4])

num = len(weights)

#Greedy Algorithm

In [168]:
coefs = np.zeros(num)
for i in range(0, num):
  coefs[i] = costs[i]/weights[i]

coefs



array([5.  , 0.5 , 1.25, 1.2 ])

In [169]:
result = np.zeros(num)

# These are here just so we don't modify the values above.
coefs2 = coefs.copy()
constraint2 = constraint

for i in range(0, num):
  maxelem = np.argmax(coefs2)
  coefs2[maxelem] = 0
  if weights[maxelem] <= constraint2:
    constraint2 -= weights[maxelem]
    result[maxelem] = 1

print("Result:")

for i in range(0, num):
  if result[i] == 1:
    print("x" + str(i+1))

print("\nWeight: " + str(constraint - constraint2))
print("\nCost: " + str(np.sum(result*costs)))




Result:
x1
x3
x4

Weight: 11

Cost: 21.0


#Branch and Bound Method



In [170]:
class Node:
  def __init__(self, coef, value, left=None, right=None):
      self.coef = coef.copy()
      self.values = value.copy()
      self.left = left
      self.right = right
      constraint2 = constraint - np.sum(weights[value == 1])
      coefs2 = self.coef
      rang = range(0, len(coef[coef!=0]))
      for i in rang:
        maxelem = np.argmax(coefs2)
        coefs2[maxelem] = 0
        if weights[maxelem] <= constraint2:
          constraint2 -= weights[maxelem]
          self.values[maxelem] = 1
        else:
          self.values[maxelem] = constraint2 / weights[maxelem]
          if len(self.values[~((self.values == 0) | (self.values == 1))]) > 0:
            newcoefs = coef.copy()
            newcoefs[maxelem] = 0
            newvalues = value.copy()
            newvalues[maxelem] = 0
            self.left = Node(newcoefs.copy(), newvalues.copy())
            newvalues[maxelem] = 1
            self.right = Node(newcoefs.copy(), newvalues.copy())
            





      

In [171]:
tabs = -1
max = None
def inorder(node):
  global tabs
  global max
  tabs += 1
  if node is None:
    tabs -= 1
    return
  inorder(node.left)
  for i in range(0, tabs):
    print("\t", end="")
  print(node.values, np.sum(node.values*costs))
  if max is None:
    maxsum = 0
  else:
    maxsum = np.sum(max.values*costs)
  if((np.sum(node.values*costs) > maxsum) & (len(node.values[~((node.values == 0) | (node.values == 1))]) == 0)):
    max = node
  inorder(node.right)
  tabs -= 1



In [172]:
root = Node(coefs.copy(), np.zeros(num))
inorder(root)

print("Result:")

for i in range(0, num):
  if max.values[i] == 1:
    print("x" + str(i+1))

print("\nWeight: " + str(np.sum(max.values*weights)))
print("\nCost: " + str(np.sum(max.values*costs)))

	[1. 0. 1. 1.] 21.0
[1.  0.5 1.  1. ] 23.0
		[1. 1. 1. 0.] 19.0
	[1.  1.  1.  0.2] 20.2
		[1. 1. 0. 1.] 20.0
Result:
x1
x3
x4

Weight: 11.0

Cost: 21.0
